In [ ]:
##!/usr/bin/env python
"""plot_AWS_comparison.py

Script plots the point time series against AWS data to evaluate model data

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu Jul 30 14:26:12 AEST 2020

"""

Load Packages

In [ ]:
#from __future__ import division
import numpy as np
import pandas as pd
import math
import netCDF4 as nc
import sys
import os
import glob as glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
from matplotlib.collections import LineCollection
import common_functions as cf
import datetime as dt
import wrf
from scipy import stats


## Model Data

In [ ]:
# Simulation Period
syear = 2017
smon = 1
sday = 2 
eyear = 2017
emon = 2
eday = 28  # Add an extra day so that the 27th Feb data is included
simlen = dt.datetime(eyear,emon,eday) - dt.datetime(syear,smon,sday)
nst = (simlen.days * 24 * 6) # No. simulations days x 24 hours in a day x 6 history intervals per hour

# Dates - Used for subsetting the AWS data so you pick the day before the start date and the day after the end date
sdate = "2017-01-01"
edate = "2017-02-28"

# Data directory 
datadir='/g/data/w97/azh561/WRF/'
ensmem = ['sydney800m','sydney800m_06H','sydney800m_12H','sydney800m_18H','sydney800m_00H']
rlabels = ['U1','U2','U3','U4','U5']
domain = ["d02","d02","d02","d02","d02"]
#domain = ["d01","d01","d01","d01","d01"]
nmem = len(ensmem)

# Landsea mask
mask_file='/g/data/w97/azh561/WRF/sydney800m_06H/geo_em.%s.nc' %(domain[0])
f = nc.Dataset(mask_file)
lu = f.variables['LU_INDEX'][0,:,:]
lat2d = f.variables['XLAT_M'][0,:,:]
lontmp = f.variables['XLONG_M'][0,:,:]
lon2d = np.where(lontmp<0.0,lontmp+360,lontmp)
clon = f.getncattr('CEN_LON')
nlu = f.getncattr('NUM_LAND_CAT')
iswater = f.getncattr('ISWATER')
f.close()

print("Domain %s Lat: %s to %s Lon: %s to %s" %(domain[0],np.nanmin(lat2d),np.nanmax(lat2d),np.nanmin(lon2d),np.nanmax(lon2d)))

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='AWS_comparison_'
if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)


## AWS Data

Data Attributes

In [ ]:
awsdir = '/g/data/w97/azh561/WRF/obs/AWS_1mindata_20stations'
awsnum = ['066037','066137','066194','067105','067108','067113','061078','061366','066062','067119','068228']
awsnm = ['Sydney Airport','Bankstown Airport','Canterbury Racecourse','Richmond RAAF','Badgerys Creek','Penrith Lakes','Williamtown RAAF','Norah Head','Sydney Observatory Hill','Horsley Park','Bellambi']
awslat = [-33.9465,-33.9176,-33.9057,-33.6004,-33.8969,-33.7195,-32.7939,-33.2814,-33.8607,-33.851,-34.3691]
awslon = [151.1731,150.9837,151.1134,150.7761,150.7281,150.6783,151.8364,151.5766,151.2050,150.8567,150.9291]
naws = len(awsnum)

Function to note the bad quality observations

In [ ]:
def get_bad_values(data):
    
    # https://stackoverflow.com/questions/19909167/how-to-find-most-frequent-string-element-in-numpy-ndarray
    unique,pos = np.unique(data,return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax() # To find the positions of the max count
    
    if unique[maxpos] in ['Y'] and counts[maxpos] == 10:
        qcflag = np.nan
    else:
        qcflag = 0.05
        
    return qcflag

Function to calculate the 10 minute average

In [ ]:
def calc_10min_avg(dataframe):
    
    # Variables of interest
    otc = dataframe['tc']
    orh = dataframe['rh']
    odp = dataframe['dp']
    omslp = dataframe['mslp']
    opr = dataframe['pr']
    owspd = dataframe['wspd']
    owdir = dataframe['wdir']

    # Quality control flags
    otc_qc = dataframe['tc_qc']
    orh_qc = dataframe['rh_qc']
    odp_qc = dataframe['dp_qc']
    omslp_qc = dataframe['mslp_qc']
    opr_qc = dataframe['pr_qc']
    owspd_qc = dataframe['wspd_qc']
    owdir_qc = dataframe['wdir_qc']    

    # Calculate the 10-minute averages - NOT THE MOST EFFICIENT WAY I'M SURE OF IT!
    odata = np.empty((7,nst),dtype=np.float64)
    oqc = np.empty((7,nst),dtype=np.float64)
    for tt in range(nst):

        # MSLP
        odata[0,tt] = np.mean([float(omslp.iloc[(tt*10)]),float(omslp.iloc[(tt*10)+1]),float(omslp.iloc[(tt*10)+2]),
             float(omslp.iloc[(tt*10)+3]),float(omslp.iloc[(tt*10)+4]),float(omslp.iloc[(tt*10)+5]),
             float(omslp.iloc[(tt*10)+6]),float(omslp.iloc[(tt*10)+7]),float(omslp.iloc[(tt*10)+8]),
             float(omslp.iloc[(tt*10)+9])])
        # T2
        odata[1,tt] = np.mean([float(otc.iloc[(tt*10)]),float(otc.iloc[(tt*10)+1]),float(otc.iloc[(tt*10)+2]),
             float(otc.iloc[(tt*10)+3]),float(otc.iloc[(tt*10)+4]),float(otc.iloc[(tt*10)+5]),
             float(otc.iloc[(tt*10)+6]),float(otc.iloc[(tt*10)+7]),float(otc.iloc[(tt*10)+8]),
             float(otc.iloc[(tt*10)+9])])
        # TD2
        odata[2,tt] = np.mean([float(odp.iloc[(tt*10)]),float(odp.iloc[(tt*10)+1]),float(odp.iloc[(tt*10)+2]),
             float(odp.iloc[(tt*10)+3]),float(odp.iloc[(tt*10)+4]),float(odp.iloc[(tt*10)+5]),
             float(odp.iloc[(tt*10)+6]),float(odp.iloc[(tt*10)+7]),float(odp.iloc[(tt*10)+8]),
             float(odp.iloc[(tt*10)+9])])
        # RH2
        odata[3,tt] = np.mean([float(orh.iloc[(tt*10)]),float(orh.iloc[(tt*10)+1]),float(orh.iloc[(tt*10)+2]),
             float(orh.iloc[(tt*10)+3]),float(orh.iloc[(tt*10)+4]),float(orh.iloc[(tt*10)+5]),
             float(orh.iloc[(tt*10)+6]),float(orh.iloc[(tt*10)+7]),float(orh.iloc[(tt*10)+8]),
             float(orh.iloc[(tt*10)+9])])
        # PR - total rather than mean
        odata[4,tt] = np.sum([float(opr.iloc[(tt*10)]),float(opr.iloc[(tt*10)+1]),float(opr.iloc[(tt*10)+2]),
             float(opr.iloc[(tt*10)+3]),float(opr.iloc[(tt*10)+4]),float(opr.iloc[(tt*10)+5]),
             float(opr.iloc[(tt*10)+6]),float(opr.iloc[(tt*10)+7]),float(opr.iloc[(tt*10)+8]),
             float(opr.iloc[(tt*10)+9])])
        # Wind Speed
        odata[5,tt] = np.mean([float(owspd.iloc[(tt*10)]),float(owspd.iloc[(tt*10)+1]),float(owspd.iloc[(tt*10)+2]),
             float(owspd.iloc[(tt*10)+3]),float(owspd.iloc[(tt*10)+4]),float(owspd.iloc[(tt*10)+5]),
             float(owspd.iloc[(tt*10)+6]),float(owspd.iloc[(tt*10)+7]),float(owspd.iloc[(tt*10)+8]),
             float(owspd.iloc[(tt*10)+9])])
        # Wind Dir
        odata[6,tt] = np.mean([float(owdir.iloc[(tt*10)]),float(owdir.iloc[(tt*10)+1]),float(owdir.iloc[(tt*10)+2]),
             float(owdir.iloc[(tt*10)+3]),float(owdir.iloc[(tt*10)+4]),float(owdir.iloc[(tt*10)+5]),
             float(owdir.iloc[(tt*10)+6]),float(owdir.iloc[(tt*10)+7]),float(owdir.iloc[(tt*10)+8]),
             float(owdir.iloc[(tt*10)+9])])

        ### Get the instances where the data quality is bad
        oqc[0,tt] = get_bad_values([omslp_qc.iloc[(tt*10)],omslp_qc.iloc[(tt*10)+1],omslp_qc.iloc[(tt*10)+2],
             omslp_qc.iloc[(tt*10)+3],omslp_qc.iloc[(tt*10)+4],omslp_qc.iloc[(tt*10)+5],
             omslp_qc.iloc[(tt*10)+6],omslp_qc.iloc[(tt*10)+7],omslp_qc.iloc[(tt*10)+8],
             omslp_qc.iloc[(tt*10)+9]])
        oqc[1,tt] = get_bad_values([otc_qc.iloc[(tt*10)],otc_qc.iloc[(tt*10)+1],otc_qc.iloc[(tt*10)+2],
             otc_qc.iloc[(tt*10)+3],otc_qc.iloc[(tt*10)+4],otc_qc.iloc[(tt*10)+5],
             otc_qc.iloc[(tt*10)+6],otc_qc.iloc[(tt*10)+7],otc_qc.iloc[(tt*10)+8],
             otc_qc.iloc[(tt*10)+9]])
        oqc[2,tt] = get_bad_values([odp_qc.iloc[(tt*10)],odp_qc.iloc[(tt*10)+1],odp_qc.iloc[(tt*10)+2],
             odp_qc.iloc[(tt*10)+3],odp_qc.iloc[(tt*10)+4],odp_qc.iloc[(tt*10)+5],
             odp_qc.iloc[(tt*10)+6],odp_qc.iloc[(tt*10)+7],odp_qc.iloc[(tt*10)+8],
             odp_qc.iloc[(tt*10)+9]])
        oqc[3,tt] = get_bad_values([orh_qc.iloc[(tt*10)],orh_qc.iloc[(tt*10)+1],orh_qc.iloc[(tt*10)+2],
             orh_qc.iloc[(tt*10)+3],orh_qc.iloc[(tt*10)+4],orh_qc.iloc[(tt*10)+5],
             orh_qc.iloc[(tt*10)+6],orh_qc.iloc[(tt*10)+7],orh_qc.iloc[(tt*10)+8],
             orh_qc.iloc[(tt*10)+9]])
        oqc[4,tt] = get_bad_values([opr_qc.iloc[(tt*10)],opr_qc.iloc[(tt*10)+1],opr_qc.iloc[(tt*10)+2],
             opr_qc.iloc[(tt*10)+3],opr_qc.iloc[(tt*10)+4],opr_qc.iloc[(tt*10)+5],
             opr_qc.iloc[(tt*10)+6],opr_qc.iloc[(tt*10)+7],opr_qc.iloc[(tt*10)+8],
             opr_qc.iloc[(tt*10)+9]])
        oqc[5,tt] = get_bad_values([owspd_qc.iloc[(tt*10)],owspd_qc.iloc[(tt*10)+1],owspd_qc.iloc[(tt*10)+2],
             owspd_qc.iloc[(tt*10)+3],owspd_qc.iloc[(tt*10)+4],owspd_qc.iloc[(tt*10)+5],
             owspd_qc.iloc[(tt*10)+6],owspd_qc.iloc[(tt*10)+7],owspd_qc.iloc[(tt*10)+8],
             owspd_qc.iloc[(tt*10)+9]])
        oqc[6,tt] = get_bad_values([owdir_qc.iloc[(tt*10)],owdir_qc.iloc[(tt*10)+1],owdir_qc.iloc[(tt*10)+2],
             owdir_qc.iloc[(tt*10)+3],owdir_qc.iloc[(tt*10)+4],owdir_qc.iloc[(tt*10)+5],
             owdir_qc.iloc[(tt*10)+6],owdir_qc.iloc[(tt*10)+7],owdir_qc.iloc[(tt*10)+8],
             owdir_qc.iloc[(tt*10)+9]])

    return odata,oqc

Read data and calculate the 10-minute averages

In [ ]:
odata = np.empty((naws,7,nst),dtype=np.float64)
oqc = np.empty((naws,7,nst),dtype=np.float64)
for ss in range(naws):
    
    # Read data
    file = "%s/HD01D_Data_%s_46163679534753.txt" %(awsdir,awsnum[ss])
    data = pd.read_csv(file)
    # MUST USE THE UTC TIME SO THAT WRF AND AWS DATA TIMES ARE THE SAME
    data.columns = ["a","No","b","c","d","e","f","g","h","i","j","k","year","month","day","hour","minute",
                "pr","pr_qc","l","tc","tc_qc","wbt","wbt_qc","dp","dp_qc","rh","rh_qc","vp","vp_qc",
                "svp","svp_qc","wspd","wspd_qc","wdir","wdir_qc","m","n","o","p","vis","vis_qc","mslp","mslp_qc","q"]

    data['date'] = pd.to_datetime(data[['year','month','day']])
    data['time'] = pd.to_datetime(data[['year','month','day','hour','minute']])

    # Clip to period of interest
    date_filter = data.loc[(data['date'] > sdate) & (data['date'] < edate)]

    # Deal with empty cells
    date_filter = date_filter.replace(r'^\s*$', np.nan, regex=True)

    # Calculate the 10 minute averages
    odata[ss,:,:],oqc[ss,:,:] = calc_10min_avg(date_filter)
    
    del data,date_filter

## Extract Model data closest to the stations - Iterate through groups of files 
    takes 40 mins per ensemble member for the 2 months of simulation
    

In [ ]:
slat = np.empty(naws,dtype=np.int)
slon = np.empty(naws,dtype=np.int)
for ss in range(naws):
    
    # Get lat/lon corresponding to the AWS site
    # https://stackoverflow.com/questions/28006077/find-i-j-location-of-closest-long-lat-values-in-a-2d-array
    a = abs(lat2d-awslat[ss])+abs(lon2d-awslon[ss])
    i0,j0 = np.unravel_index(a.argmin(),a.shape)

    slat[ss] = i0
    slon[ss] = j0
    
    del a,i0,j0
    

In [ ]:
for mm in range(nmem):

    # Files list
    filelist = sorted(glob.glob('%s/%s/WRF_output/%s/wrfout_%s_2017-*' %(datadir,ensmem[mm],domain[mm],domain[mm])))
    nfile = len(filelist)

    #for ff in range(simlen.days):
    for ff in range(int(nfile/24)):

        wrffiles = [nc.Dataset(filelist[(ff*24)]),nc.Dataset(filelist[(ff*24)+1]),nc.Dataset(filelist[(ff*24)+2])
        ,nc.Dataset(filelist[(ff*24)+3]),nc.Dataset(filelist[(ff*24)+4]),nc.Dataset(filelist[(ff*24)+5])
        ,nc.Dataset(filelist[(ff*24)+6]),nc.Dataset(filelist[(ff*24)+7]),nc.Dataset(filelist[(ff*24)+8])
        ,nc.Dataset(filelist[(ff*24)+9]),nc.Dataset(filelist[(ff*24)+10]),nc.Dataset(filelist[(ff*24)+11])
        ,nc.Dataset(filelist[(ff*24)+12]),nc.Dataset(filelist[(ff*24)+13]),nc.Dataset(filelist[(ff*24)+14])
        ,nc.Dataset(filelist[(ff*24)+15]),nc.Dataset(filelist[(ff*24)+16]),nc.Dataset(filelist[(ff*24)+17])
        ,nc.Dataset(filelist[(ff*24)+18]),nc.Dataset(filelist[(ff*24)+19]),nc.Dataset(filelist[(ff*24)+20])
        ,nc.Dataset(filelist[(ff*24)+21]),nc.Dataset(filelist[(ff*24)+22]),nc.Dataset(filelist[(ff*24)+23])]

        # Extract the variables of interest
        timetmp  = wrf.getvar(wrffiles,"times",timeidx=None,method='cat')               # Times
        rh2tmp   = wrf.getvar(wrffiles,"rh2",timeidx=None,method='cat')[:,slat,slon]                         # 2m Relative Humidity
        td2tmp   = wrf.getvar(wrffiles,"td2",units='degC',timeidx=None,method='cat')[:,slat,slon]            # 2m Dew Point Temperature
        t2tmp    = wrf.getvar(wrffiles,'T2',timeidx=None,method='cat')[:,slat,slon] - 273.15                               # 2m temperature
        psfctmp  = wrf.getvar(wrffiles,'PSFC',timeidx=None,method='cat')[:,slat,slon] /100.                                # surface pressure hPa
        prtmp    = wrf.getvar(wrffiles,'RAINC',timeidx=None,method='cat')[:,slat,slon] + wrf.getvar(wrffiles,'RAINNC',timeidx=None,method='cat')[:,slat,slon] # total precipitation mm
        windtmp  = wrf.getvar(wrffiles,"wspd_wdir10",units='km h-1',timeidx=None,method='cat')[:,:,slat,slon] # 10m wind speed and direction

        # Append to arrays
        if ff == 0:
            ftimes = timetmp
            rh2 = rh2tmp
            td2 = td2tmp
            t2 = t2tmp
            psfc = psfctmp
            pr = prtmp
            wspd = windtmp[0,:,:,:]
            wdir = windtmp[1,:,:,:]
        else:
            ftimes = np.append(ftimes,timetmp,axis=0)
            rh2 = np.append(rh2,rh2tmp,axis=0)
            td2 = np.append(td2,td2tmp,axis=0)
            t2 = np.append(t2,t2tmp,axis=0)
            psfc = np.append(psfc,psfctmp,axis=0)
            pr = np.append(pr,prtmp,axis=0)
            wspd = np.append(wspd,windtmp[0,:,:,:],axis=0)
            wdir = np.append(wdir,windtmp[1,:,:,:],axis=0)

        # Cleanup
        del timetmp,rh2tmp,td2tmp,t2tmp,psfctmp,prtmp,windtmp

    ftimes = ftimes.astype('datetime64[m]')

    if mm == 0:
        tsdata = np.empty((naws,nmem,7,len(ftimes)),dtype=np.float64) # [nmem+1,nvar,ntime]
        times = ftimes

    for ss in range(naws):
        tsdata[ss,mm,0,:len(ftimes)] = psfc[:len(ftimes),ss,ss]
        tsdata[ss,mm,1,:len(ftimes)] = t2[:len(ftimes),ss,ss]
        tsdata[ss,mm,2,:len(ftimes)] = td2[:len(ftimes),ss,ss]
        tsdata[ss,mm,3,:len(ftimes)] = rh2[:len(ftimes),ss,ss]
        tsdata[ss,mm,4,:len(ftimes)] = 0.0 # First set all pr values to zero
        for tt in range(len(pr)-1):
            tsdata[ss,mm,4,tt] = pr[tt+1,ss,ss] - pr[tt,ss,ss]
        tsdata[ss,mm,5,:len(ftimes)] = wspd[:len(ftimes),ss,ss]
        tsdata[ss,mm,6,:len(ftimes)] = wdir[:len(ftimes),ss,ss]

    # Once data read for an ensemble member - write to file - saves read time later!
    for ss in range(naws):
        datadump = np.vstack([tsdata[ss,mm,0,:len(ftimes)], tsdata[ss,mm,1,:len(ftimes)], tsdata[ss,mm,2,:len(ftimes)],
         tsdata[ss,mm,3,:len(ftimes)], tsdata[ss,mm,4,:len(ftimes)], tsdata[ss,mm,5,:len(ftimes)],
            tsdata[ss,mm,6,:len(ftimes)]])
        np.savetxt('WRF_output_M%s_%s_AWS%s.txt' %(ensmem[mm],domain[mm],awsnum[ss]), (datadump.T), delimiter = ' ',header = "PSFC T2 TD2 RH2 PR WSPD WDIR", fmt = '%0.4f %0.4f %0.4f %0.4f %0.4f %0.4f %0.4f')
        del datadump
        
    del psfc,t2,td2,rh2,pr,wspd,wdir,filelist,nfile


# Read in the previously extracted data

In [ ]:
start = dt.datetime(syear,smon,sday,0,0,0)
end = dt.datetime(eyear,emon,eday,0,0,0)
days = (end - start).days
ntim = days * 24 * 60
datelist = [start + dt.timedelta(minutes=x) for x in range(ntim+1)]
# Get the day-month hour-minutes on 10 minute interval
ftimes = np.asarray([datelist[x].strftime("%m-%d %H-%M") for x in range(ntim+1)])[::10]
fdates = np.asarray([datelist[x].strftime("%m-%d") for x in range(ntim+1)])[::10]

In [ ]:
tsdata = np.empty((naws,nmem,7,len(ftimes)),dtype=np.float64)
# Loop through the ensemble members
for mm in range(nmem):

    # Loop through the sites
    for ss in range(naws):
    
        # Read data
        data = pd.read_csv('WRFOUT_AWS_EXTRACTED_GRIDS/WRF_output_M%s_%s_AWS%s.txt' %(ensmem[mm],domain[mm],awsnum[ss]),delimiter = ' ')
        data.columns = ["PSFC", "T2", "TD2", "RH2", "PR", "WSPD", "WDIR","#"]
    
        tsdata[ss,mm,0,:] = data['PSFC'].iloc[0:len(ftimes)]
        tsdata[ss,mm,1,:] = data['T2'].iloc[0:len(ftimes)]
        tsdata[ss,mm,2,:] = data['TD2'].iloc[0:len(ftimes)]
        tsdata[ss,mm,3,:] = data['RH2'].iloc[0:len(ftimes)]
        tsdata[ss,mm,4,:] = data['PR'].iloc[0:len(ftimes)]
        tsdata[ss,mm,5,:] = data['WSPD'].iloc[0:len(ftimes)]
        tsdata[ss,mm,6,:] = data['WDIR'].iloc[0:len(ftimes)]
    
        del data

## Dealing with time files of the WRF data

In [ ]:
#spind = [i for i in range(len(times)) if times[i] in [np.datetime64('2017-02-28T00:00')]][0]
spind = [i for i in range(len(ftimes)) if ftimes[i] in ['02-28 00-00']][0]

## Plot the data

In [ ]:
# Function to plot data
def plot_ts(time,tsdata,odata,oqc,rlabels,vlabels,mtitle,figurename,lspace):

    """This function plots time series for observations and models"""

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    import string
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=500
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    plt.rcParams["font.size"] = 18

    # Define dimensions
    nmod = tsdata.shape[0]
    nvar = tsdata.shape[1]
    nt = tsdata.shape[2]
      
    # Create figure object and subplots
    fig, ax = plt.subplots(nvar, 1, figsize=(30.0,5.0*(nvar)), squeeze=False)
    tarr = np.arange(0,nt)
    
    colors = ["red","blue","green","orange","purple"] 
    ypos = [0.9,0.8,0.7,0.6,0.5]
    
    # Iterate through variables
    for vind in range(nvar):

        # Observations
        ax[vind,0].plot(tarr,odata[vind,:],linestyle='-',linewidth=4,color='black',label='AWS')

        # Add the observation flags where the is poor data quality
        ax2 = ax[vind,0].twinx()
        ax2.plot(tarr,oqc[vind,:],marker='o',markersize=3,linestyle = 'None',color='DarkGrey')
        ax2.set_ylim(0,1)
        ax2.set_yticks([],[])
        ax2.set_xticks([],[])
        ax2.set_xlim(tarr[0],tarr[-1])

        # Models
        for mind in range(nmod):
            if mind == 2:
                xdata = tarr[:len(ftimes)]
                ydata = tsdata[mind,vind,:len(ftimes)]
                oy = odata[vind,:len(ftimes)]
            else:
                xdata = tarr
                ydata = tsdata[mind,vind,:]
                oy = odata[vind,:]
            ax[vind,0].plot(xdata,ydata, linewidth=2,color=colors[mind], linestyle='-', label=rlabels[mind])
            # Add statistics
            bias = np.round(cf.calc_bias(ydata,oy,lat2d,flag=False),2)
            rmse = np.round(cf.calc_rmse(ydata,oy,lat2d,flag=False),2)
            phase = np.round(cf.calc_phase(ydata,oy,lat2d,flag=False),2)
            ax[vind,0].text(0.07,ypos[mind],'$S_{B}$=%s, $S_{R}$=%s, $S_{P}$=%s' %(bias,rmse,phase), 
                            horizontalalignment='center',verticalalignment='center',transform = ax[vind,0].transAxes,
                            color=colors[mind], fontweight='bold', fontsize=18)
            del bias,rmse,phase
            del xdata,ydata,oy

        # Fix Labelling
        ax[vind,0].set_ylabel('%s' %(vlabels[vind]), fontweight = 'bold',fontsize=20)
 
        # Amend axis limits
        ax[vind,0].set_xlim(tarr[0]-1300,tarr[-1])
        
        if vind < nvar-1:
            ax[vind,0].set_xticks([],[])
        else:
            ax[vind,0].set_xticks(tarr[::lspace])
            ax[vind,0].set_xticklabels(time[::lspace],rotation=90,fontsize=18)

    ax[0,0].set_title(mtitle, fontweight = 'bold',fontsize=20)
    legend = ax[-1,0].legend(loc='upper center', bbox_to_anchor=(0.5,-0.275), ncol=6, fontsize=20)

    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(figurename,bbox_extra_artists=(legend,), bbox_inches='tight')
    plt.close(fig)


In [ ]:
vlabels = ['MSLP [hPa]','$T_{2m}$ [\xb0 C]','$T_{dp}$ [\xb0 C]', 'RH [%]','PR [mm]','WSPD [$km.hr^{-1}$]','WDIR [\xb0]']
lspace = 144 # As the wrf output is saved at a 10 minute interval

for ss in range(naws):
    figurename = 'Times_Series_Validation_%s.png' %(awsnm[ss])
    plot_ts(fdates[:spind],tsdata[ss,:,:,:spind],odata[ss,:,:spind],oqc[ss,:,:spind],rlabels,vlabels,awsnm[ss],figurename,lspace)


In [ ]:
# For d01 domain

vlabels = ['MSLP [hPa]','$T_{2m}$ [\xb0 C]','$T_{dp}$ [\xb0 C]', 'RH [%]','PR [mm]','WSPD [$km.hr^{-1}$]','WDIR [\xb0]']
lspace = 144 # As the wrf output is saved at a 10 minute interval
spind = 8200

for ss in range(naws):
    figurename = 'Times_Series_Validation_d01_%s.png' %(awsnm[ss])
    plot_ts(ftimes[:spind],tsdata[ss,:,:,:spind],odata[ss,:,:spind],oqc[ss,:,:spind],rlabels,vlabels,awsnm[ss],figurename,lspace)


Calculate the statistics of the ensemble mean

In [ ]:
vlabels = ['MSLP [hPa]','$T_{2m}$ [\xb0 C]','$T_{dp}$ [\xb0 C]', 'RH [%]','PR [mm]','WSPD [$km.hr^{-1}$]','WDIR [\xb0]']
for ss in range(naws):
    print(awsnm[ss])
    tsmean = np.nanmean(tsdata[ss,:,:,:spind],axis=0)
    
    for vind in range(7):
        print(vlabels[vind])
        bias = np.round(cf.calc_bias(tsmean[vind,:],odata[ss,vind,:spind],lat2d,flag=False),2)
        rmse = np.round(cf.calc_rmse(tsmean[vind,:],odata[ss,vind,:spind],lat2d,flag=False),2)
        phase = np.round(cf.calc_phase(tsmean[vind,:],odata[ss,vind,:spind],lat2d,flag=False),2) 
        
        print([bias,rmse,phase])
        del bias,rmse,phase
        
    del tsmean

## Contents of the AWS file
    1  hd
    2  Station Number
#### datetime format in Local time
    3  YYYY
    4  MM 
    5  DD
    6  HH24
    7  MI
#### datetime format in Local standard time
    8  YYYY
    9  MM 
    10 DD
    11 HH24
    12 MI
#### datetime format in Universal coordinated time
    13 YYYY
    14 MM 
    15 DD
    16 HH24
    17 MI
#### variables
    18 Precipitation since last (AWS) observation in mm
    19 Quality of precipitation since last (AWS) observation value
    20 Period over which precipitation since last (AWS) observation is measured in minutes
    21 Air Temperature in degrees Celsius
    22 Quality of air temperature
    23 Wet bulb temperature in degrees Celsius
    24 Quality of Wet bulb temperature
    25 Dew point temperature in degrees Celsius
    26 Quality of dew point temperature
    27 Relative humidity in percentage %
    28 Quality of relative humidity
    29 Vapour pressure in hPa
    30 Quality of vapour pressure
    31 Saturated vapour pressure in hPa
    32 Quality of saturated vapour pressure
    33 Wind (1 minute) speed in km/h,
    34 Wind (1 minute) speed quality
    35 Wind (1 minute) direction in degrees true
    36 Wind (1 minute) direction quality
    37 Standard deviation of wind (1 minute) 
    38 Standard deviation of wind (1 minute) direction quality
    39 Maximum wind gust (over 1 minute) in km/h
    40 Maximum wind gust (over 1 minute) quality
    41 Visibility (automatic - one minute data) in km
    42 Quality of visibility (automatic - one minute data) 
    43 Mean sea level pressure in hPa
    44 Quality of mean sea level pressure
